# Pipeline Profiling

Analyze where time is spent in the experiment pipeline.

Data sources:
- `results.json` → `metadata.timing` (phase + metric durations)
- `predictions.json` → `metric_timings` (per-metric durations)
- `experiment.log` → parsed for model load/inference split

In [ ]:
import json
import re
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

STUDY_DIR = Path("../outputs/smart_retrieval_slm")

SKIP_DIRS = {"_archived_fake_reranked", "_archived_old_names", "_tainted",
             "_collisions", "_incomplete", "_quarantined", "analysis",
             "__pycache__", ".ipynb_checkpoints"}

## 1. Load timing data from results.json

In [ ]:
records = []

for exp_dir in sorted(STUDY_DIR.iterdir()):
    if not exp_dir.is_dir() or exp_dir.name in SKIP_DIRS or exp_dir.name.startswith((".", "_")):
        continue

    results_path = exp_dir / "results.json"
    if not results_path.exists():
        continue

    with open(results_path) as f:
        data = json.load(f)

    meta = data.get("metadata", {})
    timing = meta.get("timing", {})
    phases = timing.get("phases", {})
    metric_t = timing.get("metrics", {})

    # Also try predictions.json for metric_timings (works for older experiments)
    if not metric_t:
        preds_path = exp_dir / "predictions.json"
        if preds_path.exists():
            try:
                with open(preds_path) as f:
                    pdata = json.load(f)
                metric_t = pdata.get("metric_timings", {})
            except Exception:
                pass

    rec = {
        "name": exp_dir.name,
        "exp_type": meta.get("exp_type", meta.get("type", "unknown")),
        "model": (meta.get("model", "").split("/")[-1] if "/" in meta.get("model", "") 
                  else meta.get("model", "unknown")),
        "dataset": meta.get("dataset", "unknown"),
        "retriever": meta.get("retriever", "none"),
        "agent_type": meta.get("agent_type", "fixed_rag"),
        "total_s": data.get("duration_seconds", 0),
        "f1": data.get("metrics", {}).get("f1", 0),
        # Phase timings
        "phase_init": phases.get("init", 0),
        "phase_generating": phases.get("generating", 0),
        "phase_generated": phases.get("generated", 0),
        "phase_metrics": phases.get("computing_metrics", 0),
        "phase_complete": phases.get("complete", 0),
        # Metric timings
        "metric_f1": metric_t.get("f1", 0),
        "metric_exact_match": metric_t.get("exact_match", 0),
        "metric_bertscore": metric_t.get("bertscore", 0),
        "metric_bleurt": metric_t.get("bleurt", 0),
    }
    records.append(rec)

df = pd.DataFrame(records)
print(f"Loaded {len(df)} experiments")
print(f"  With timing data: {(df['phase_generating'] > 0).sum()}")
print(f"  Without timing data: {(df['phase_generating'] == 0).sum()} (pre-profiling runs)")
df.head()

## 2. Parse experiment.log for model load vs inference split

For experiments without structured timing data, fall back to parsing logs.

In [ ]:
def parse_experiment_log(log_path: Path) -> dict:
    """Extract timing info from experiment.log."""
    result = {"model_load_s": 0, "inference_s": 0, "phase_timings": {}, "metric_timings": {}}
    if not log_path.exists():
        return result

    text = log_path.read_text(errors="replace")

    # Generator loaded in 28.0s
    m = re.search(r"Generator loaded in ([\d.]+)s", text)
    if m:
        result["model_load_s"] = float(m.group(1))

    # Step [batch_generate] ... completed in 2.5s
    m = re.search(r"Step \[batch_generate\].*completed in ([\d.]+)s", text)
    if m:
        result["inference_s"] = float(m.group(1))

    # Phase timings: Phase [generating] completed in 32.2s
    for m in re.finditer(r"Phase \[(\w+)\] completed in ([\d.]+)s", text):
        result["phase_timings"][m.group(1)] = float(m.group(2))

    # Metric timings: bertscore done in 15.3s  (new format)
    for m in re.finditer(r"(\w+) done in ([\d.]+)s", text):
        result["metric_timings"][m.group(1)] = float(m.group(2))

    return result


# Enrich dataframe with log-parsed timings for experiments missing structured data
log_records = []
for _, row in df.iterrows():
    log_path = STUDY_DIR / row["name"] / "experiment.log"
    parsed = parse_experiment_log(log_path)
    log_records.append({
        "name": row["name"],
        "model_load_s": parsed["model_load_s"],
        "inference_s": parsed["inference_s"],
    })

    # Backfill phase timings from log if structured data is missing
    if row["phase_generating"] == 0 and parsed["phase_timings"]:
        idx = df.index[df["name"] == row["name"]]
        for phase, t in parsed["phase_timings"].items():
            col = f"phase_{phase}"
            if col in df.columns:
                df.loc[idx, col] = t

    # Backfill metric timings from log if missing
    if row["metric_bertscore"] == 0 and parsed["metric_timings"]:
        idx = df.index[df["name"] == row["name"]]
        for metric, t in parsed["metric_timings"].items():
            col = f"metric_{metric}"
            if col in df.columns:
                df.loc[idx, col] = t

df_logs = pd.DataFrame(log_records)
df = df.merge(df_logs, on="name", how="left")

print(f"Log-parsed: {(df['model_load_s'] > 0).sum()} experiments with model load times")
df[["name", "total_s", "model_load_s", "inference_s", "phase_generating", "phase_metrics"]].head(10)

## 3. Phase breakdown

In [ ]:
# Filter to experiments with timing data
timed = df[df["phase_generating"] > 0].copy()

if len(timed) == 0:
    print("No experiments with structured timing data yet.")
    print("Run experiments with the updated code to collect timing data.")
    print("\nFalling back to log-parsed data...")
    timed = df[df["model_load_s"] > 0].copy()
    if len(timed) > 0:
        # Use log-parsed phase timings
        timed["phase_other"] = timed["total_s"] - timed["phase_generating"] - timed["phase_metrics"]
        timed["phase_other"] = timed["phase_other"].clip(lower=0)

if len(timed) > 0:
    # Average phase breakdown by experiment type
    phase_cols = ["phase_generating", "phase_metrics", "phase_init", "phase_generated", "phase_complete"]
    available_cols = [c for c in phase_cols if c in timed.columns and timed[c].sum() > 0]

    by_type = timed.groupby("exp_type")[available_cols + ["total_s"]].mean()

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Bar chart: average time per phase
    by_type[available_cols].plot.bar(stacked=True, ax=axes[0], colormap="Set2")
    axes[0].set_title("Average phase breakdown by experiment type")
    axes[0].set_ylabel("Time (seconds)")
    axes[0].tick_params(axis="x", rotation=0)
    axes[0].legend(loc="upper left", fontsize=8)

    # Pie chart: overall time allocation
    totals = timed[available_cols].sum()
    labels = [c.replace("phase_", "") for c in available_cols]
    axes[1].pie(totals, labels=labels, autopct="%1.1f%%", startangle=90)
    axes[1].set_title(f"Overall time allocation ({len(timed)} experiments)")

    plt.tight_layout()
    plt.show()
else:
    print("No timing data available. Run experiments first.")

## 4. Metric computation breakdown

In [ ]:
metric_cols = [c for c in df.columns if c.startswith("metric_") and df[c].sum() > 0]

if metric_cols:
    m_timed = df[df[metric_cols].sum(axis=1) > 0].copy()
    avg_metric = m_timed[metric_cols].mean().sort_values(ascending=False)

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Bar: average metric computation time
    labels = [c.replace("metric_", "") for c in avg_metric.index]
    axes[0].barh(labels, avg_metric.values, color="steelblue")
    axes[0].set_xlabel("Time (seconds)")
    axes[0].set_title(f"Average metric computation time ({len(m_timed)} experiments)")
    for i, v in enumerate(avg_metric.values):
        axes[0].text(v + 0.2, i, f"{v:.1f}s", va="center", fontsize=9)

    # Proportion pie
    totals = m_timed[metric_cols].sum()
    labels_pie = [c.replace("metric_", "") for c in totals.index]
    axes[1].pie(totals, labels=labels_pie, autopct="%1.1f%%", startangle=90)
    axes[1].set_title("Metric time share")

    plt.tight_layout()
    plt.show()
else:
    print("No per-metric timing data yet. Run experiments with updated code.")

## 5. Model load vs inference time

In [ ]:
with_load = df[(df["model_load_s"] > 0) & (df["inference_s"] > 0)].copy()

if len(with_load) > 0:
    with_load["overhead_s"] = (
        with_load["phase_generating"] - with_load["model_load_s"] - with_load["inference_s"]
    ).clip(lower=0)

    by_model = with_load.groupby("model")[["model_load_s", "inference_s", "overhead_s"]].mean()
    by_model = by_model.sort_values("model_load_s", ascending=True)

    fig, ax = plt.subplots(figsize=(12, max(4, len(by_model) * 0.6)))
    by_model.plot.barh(stacked=True, ax=ax, color=["#e74c3c", "#2ecc71", "#95a5a6"])
    ax.set_xlabel("Time (seconds)")
    ax.set_title("Generation phase breakdown by model")
    ax.legend(["Model load", "Inference", "Other overhead"])
    plt.tight_layout()
    plt.show()

    # Print summary table
    summary = with_load.groupby("model").agg(
        count=("name", "count"),
        avg_load=("model_load_s", "mean"),
        avg_infer=("inference_s", "mean"),
        avg_total=("total_s", "mean"),
    ).round(1)
    summary["load_pct"] = (summary["avg_load"] / summary["avg_total"] * 100).round(1)
    print("\nModel load as % of total time:")
    display(summary.sort_values("avg_load", ascending=False))
else:
    print("No model load timing data found in experiment logs.")

## 6. Time per experiment type and model

In [ ]:
if len(df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # By experiment type
    by_type = df.groupby("exp_type")["total_s"].agg(["mean", "count"])
    by_type["mean"].plot.bar(ax=axes[0], color="steelblue")
    axes[0].set_title("Average total time by experiment type")
    axes[0].set_ylabel("Time (seconds)")
    axes[0].tick_params(axis="x", rotation=0)
    for i, (_, row) in enumerate(by_type.iterrows()):
        axes[0].text(i, row["mean"] + 1, f"n={int(row['count'])}", ha="center", fontsize=9)

    # By model
    by_model = df.groupby("model")["total_s"].agg(["mean", "count"]).sort_values("mean")
    by_model["mean"].plot.barh(ax=axes[1], color="coral")
    axes[1].set_title("Average total time by model")
    axes[1].set_xlabel("Time (seconds)")
    for i, (_, row) in enumerate(by_model.iterrows()):
        axes[1].text(row["mean"] + 1, i, f"n={int(row['count'])}", va="center", fontsize=9)

    plt.tight_layout()
    plt.show()

## 7. Bottleneck summary

In [ ]:
print("=" * 70)
print("BOTTLENECK SUMMARY")
print("=" * 70)

if len(df) > 0:
    total_hours = df["total_s"].sum() / 3600
    print(f"\nTotal compute time: {total_hours:.1f} hours across {len(df)} experiments")
    print(f"Average per experiment: {df['total_s'].mean():.0f}s")
    print(f"Median per experiment: {df['total_s'].median():.0f}s")

    if df["phase_generating"].sum() > 0:
        gen_total = df["phase_generating"].sum()
        metrics_total = df["phase_metrics"].sum()
        total = gen_total + metrics_total
        if total > 0:
            print(f"\nPhase split (of generation + metrics):")
            print(f"  Generation: {gen_total/3600:.1f}h ({gen_total/total*100:.0f}%)")
            print(f"  Metrics:    {metrics_total/3600:.1f}h ({metrics_total/total*100:.0f}%)")

    if df["model_load_s"].sum() > 0:
        load_total = df["model_load_s"].sum()
        infer_total = df["inference_s"].sum()
        print(f"\nGeneration breakdown (from logs):")
        print(f"  Model loading: {load_total/3600:.1f}h ({load_total/df['total_s'].sum()*100:.0f}% of total)")
        print(f"  Inference:     {infer_total/3600:.1f}h ({infer_total/df['total_s'].sum()*100:.0f}% of total)")

    if metric_cols:
        m_timed = df[df[metric_cols].sum(axis=1) > 0]
        if len(m_timed) > 0:
            print(f"\nMetric breakdown (avg per experiment):")
            for col in metric_cols:
                avg = m_timed[col].mean()
                if avg > 0:
                    name = col.replace("metric_", "")
                    print(f"  {name:20s} {avg:6.1f}s")

    # Slowest experiments
    print(f"\nSlowest 5 experiments:")
    for _, row in df.nlargest(5, "total_s").iterrows():
        print(f"  {row['name']:45s} {row['total_s']:6.0f}s ({row['exp_type']})")

print("\n" + "=" * 70)